In [ ]:
from PIL import Image
import glob
import torchvision
from matplotlib import pyplot as plt
import os
import torch



import torchvision.transforms as transforms

In [161]:



transform = transforms.Compose([
                                # transforms.PILToTensor(),
                                transforms.ToTensor(),
                                transforms.Resize((128,128))])




main_path='/local/scratch/a/rahman89/PycharmProjects/STAR_GAN/IDDAG/original_celeba_128'
star_path='/local/scratch/a/rahman89/PycharmProjects/STAR_GAN/IDDAG/stargan_celeba_128'
egdse_path='/local/scratch/a/rahman89/PycharmProjects/STAR_GAN/IDDAG/egsde_celeba_256'


main_list= []
star_list = []
egdse_list=[]


num_images=848
for iter in range(num_images):
    fname0= f'{main_path}/img{iter}.png'
    fname1= f'{star_path}/img{iter}.png'
    fname2= f'{egdse_path}/img{iter}.png'


    if os.path.isfile(fname0) and os.path.isfile(fname1) and os.path.isfile(fname2):

        im=Image.open(fname0)
        im= transform(im).to('cuda')
        main_list.append(im.unsqueeze(0))

        im=Image.open(fname1)
        im= transform(im).to('cuda')
        star_list.append(im.unsqueeze(0))

        im=Image.open(fname2)
        im= transform(im).to('cuda')
        egdse_list.append(im.unsqueeze(0))








/local/scratch/a/rahman89/PycharmProjects/IDDAG/venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [162]:
len(egdse_list)

848

In [163]:
main_images = torch.cat(main_list)
stargan_images = torch.cat(star_list)
egsde_images = torch.cat(egdse_list)


In [132]:
len(egsde_images)

420

In [ ]:

import numpy as np


def plot_image_ara(img_ara, folder=None, title=None):
    rows=img_ara.shape[0]
    cols=img_ara.shape[1]

    print(rows,cols)

    f, axarr = plt.subplots(rows, cols, figsize=(cols, rows), squeeze=False)
    for c in range(cols):

        if c==0:
            # axarr[r, c].get_yaxis().set_ticks([ '','x', ''])
            # axarr[r, c].get_yaxis().set_ticks([x for x in range(-10,11)])
            axarr[0, c].set_yticklabels( ['', '', 'Original'], rotation=45 ,fontsize=12 )
            axarr[1, c].set_yticklabels( ['', '', 'StarGAN'], rotation=45 ,fontsize=12)
            axarr[2, c].set_yticklabels( ['', '', 'EGDSE'], rotation=45 ,fontsize=12)

        for r in range(rows):
            if c>0:
                axarr[r, c].get_yaxis().set_ticks([])

            axarr[r, c].get_xaxis().set_ticks([])
            img= img_ara[r][c].cpu().detach().numpy()
            img= np.transpose(img, (1,2,0))
            axarr[r, c].imshow(img)


        f.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0, hspace=0)

    if folder==None:

        plt.show()
    else:
        os.makedirs(folder, exist_ok=True)
        plt.savefig(f'{folder}/{title}.png', bbox_inches='tight')

    plt.close()

In [ ]:

for st in range(200,300,10):
# st=120
    en= st+8

    all_img= torch.cat([main_images[st:en].unsqueeze(0), stargan_images[st:en].unsqueeze(0), egsde_images[st:en].unsqueeze(0)])
    # plot_image_ara(all_img)
    plot_image_ara(all_img,'/local/scratch/a/rahman89/PycharmProjects/STAR_GAN/IDDAG/PLOTS/',f'comparison{st}_{en}')



In [113]:


from IDDAG.constantFunctions import get_classifier

label_path = "/local/scratch/a/rahman89/Datasets/CelebAMask-HQ/CelebAMask-HQ-attribute-anno.txt"
attributes = open(label_path).readlines()[1].split(' ')
attributes[-1] = attributes[-1].strip('\n')
classifier, trainer = get_classifier(attributes, IMAGE_SIZE=128)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Classifier loaded


/local/scratch/a/rahman89/PycharmProjects/IDDAG/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [164]:
from torch.utils.data import DataLoader

def get_prediction(classifier, trainer, images):

    transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ]
    )

    data_list = []
    for img in images:
        lbl = torch.zeros(40, 1)
        data_list.append([transform(img), lbl])

    predict_loader = DataLoader(dataset=data_list, batch_size=1, shuffle=False)
    prediction = trainer.predict(classifier, predict_loader)  # without fine-tuning
    all=[]
    for idx, data_input in enumerate(prediction):
        pred= data_input[2][0]
        all.append(pred)
    return all




In [165]:
pred_main = get_prediction(classifier, trainer, main_images)
pred_star = get_prediction(classifier, trainer, stargan_images)
pred_egsde = get_prediction(classifier, trainer, egsde_images)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/local/scratch/a/rahman89/PycharmProjects/IDDAG/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [166]:


####
def get_new_att(pred1, pred2):
    increased =  {}
    for lb in attributes:
        increased[lb] = 0

    for st, en in zip(pred1, pred2):
        diff = set(en) - set(st)
        for lb in diff:
            increased[lb] += 1

    for lb in increased:
        increased[lb] = increased[lb]/ (len(pred1))*100
    increased =dict(sorted(increased.items(), key=lambda item: item[1]))

    return increased

x= get_new_att(pred_main, pred_star)
y= get_new_att(pred_main, pred_egsde)


x = dict(reversed(x.items()))
y = dict(reversed(y.items()))

In [167]:
x

{'Wearing_Lipstick': 33.72641509433962,
 'No_Beard': 26.650943396226417,
 'High_Cheekbones': 20.28301886792453,
 'Arched_Eyebrows': 20.28301886792453,
 'Heavy_Makeup': 18.160377358490564,
 'Receding_Hairline': 12.382075471698114,
 'Attractive': 11.556603773584905,
 'Black_Hair': 10.731132075471699,
 'Pointy_Nose': 9.19811320754717,
 'Blurry': 7.783018867924528,
 'Young': 6.721698113207547,
 'Smiling': 6.25,
 'Oval_Face': 5.89622641509434,
 'Bags_Under_Eyes': 5.89622641509434,
 'Mouth_Slightly_Open': 5.660377358490567,
 'Gray_Hair': 5.306603773584905,
 'Brown_Hair': 4.952830188679245,
 'Wearing_Earrings': 2.5943396226415096,
 'Narrow_Eyes': 2.1226415094339623,
 'Big_Nose': 2.1226415094339623,
 'Bangs': 2.1226415094339623,
 'Blond_Hair': 1.2971698113207548,
 'Wavy_Hair': 1.179245283018868,
 'Straight_Hair': 0.9433962264150944,
 'Eyeglasses': 0.7075471698113208,
 'Double_Chin': 0.7075471698113208,
 'Big_Lips': 0.589622641509434,
 'Wearing_Hat': 0.3537735849056604,
 'Chubby': 0.35377358490

In [168]:
y

{'Wearing_Lipstick': 81.60377358490565,
 'Heavy_Makeup': 68.27830188679245,
 'Arched_Eyebrows': 45.75471698113208,
 'Oval_Face': 41.863207547169814,
 'High_Cheekbones': 36.556603773584904,
 'Attractive': 36.20283018867924,
 'Wearing_Earrings': 33.9622641509434,
 'No_Beard': 27.830188679245282,
 'Young': 26.41509433962264,
 'Mouth_Slightly_Open': 21.34433962264151,
 'Smiling': 19.339622641509436,
 'Straight_Hair': 16.27358490566038,
 'Rosy_Cheeks': 11.556603773584905,
 'Pointy_Nose': 11.43867924528302,
 'Receding_Hairline': 10.141509433962264,
 'Bangs': 8.962264150943396,
 'Blond_Hair': 8.372641509433961,
 'Brown_Hair': 7.547169811320755,
 'Big_Lips': 4.009433962264151,
 'Wavy_Hair': 3.4198113207547167,
 'Black_Hair': 3.0660377358490565,
 'Big_Nose': 2.358490566037736,
 'Double_Chin': 1.4150943396226416,
 'Bags_Under_Eyes': 1.4150943396226416,
 'Eyeglasses': 1.179245283018868,
 'Wearing_Necklace': 0.7075471698113208,
 'Narrow_Eyes': 0.7075471698113208,
 'Gray_Hair': 0.7075471698113208,
